**List of Noteable Variables in this Workbook**

- combined_df : dataset dataframe after removing duplication 
- drop_df : dataset dataframe after removing duplication + missing value 
- drop_df_removed_shift : dataset dataframe after removing duplication + missing value + shifted rows

## Import Data & Data Preprocessing

In [1]:
import pandas as pd

In [2]:
import numpy as np
import os

In [4]:
#pip install pyarrow
#pip install fastparquet

**1. Remove Duplication**

Step 1: remove duplicate iteration  
(third line of code) The variable "files" inside pd.read_parquet() of the original file is revised to "file" .

In [53]:
num_files = 15
files = [f"files/batch_{i}.parquet" for i in range(num_files)]

dfs = [pd.read_parquet(file) for file in files]
combined_df = pd.concat(dfs, ignore_index=True)
combined_df

,Investors,Primary Contact,Description,Geography,Preferred Industry,Preferred Investment Type,Primary Investor Type,geography_tags,preferred_investment_type_tags,preferred_industry_tags
0,Techstars,David Cohen,"Founded in 26, Techstars is an accelerator bas...","Africa, Americas, Asia, Canada, Middle East, O...","Beverages, Computer Hardware, Education and Tr...","Accelerator/Incubator, Early Stage VC, Later S...",Accelerator/Incubator,"[africa, americas, asia, canada, middle east, ...","[accelerator/incubator, early stage vc, later ...","[beverages, computer hardware, education and t..."
1,Y Combinator,David Lieb,"Founded in 25, Y Combinator is an accelerator ...","Africa, Americas, Asia, Europe, Oceania, Unite...","Biotechnology, Commercial Transportation, Comm...","Accelerator/Incubator, Early Stage VC, Later S...",Accelerator/Incubator,"[africa, americas, asia, europe, oceania, unit...","[accelerator/incubator, early stage vc, later ...","[biotechnology, commercial transportation, com..."
2,Plug and Play Tech Center,Marc Steiner,"Founded in 26, Plug and Play Tech Center is an...","Africa, Americas, Asia, Canada, Europe, Middle...","Aerospace and Defense, Animal Husbandry, Aquac...","Accelerator/Incubator, Early Stage VC, Later S...",Accelerator/Incubator,"[africa, americas, asia, canada, europe, middl...","[accelerator/incubator, early stage vc, later ...","[aerospace and defense, animal husbandry, aqua..."
3,Gaingels,Paul Grossinger,"Founded in 214, Gaingels is a venture capital ...","Africa, Americas, Asia, Canada, Europe, Middle...","Business Products and Services (B2B), Consumer...","Early Stage VC, Later Stage VC, PE Growth/Expa...",Venture Capital,"[africa, americas, asia, canada, europe, middl...","[early stage vc, later stage vc, pe growth/exp...","[business products and services (b2b), consume..."
4,Antler,Magnus Grimeland,"Founded in 217, Antler is a venture capital in...","Australia, Brazil, Canada, China, Denmark, Est...","Agriculture, Business Products and Services (B...","Accelerator/Incubator, Early Stage VC, Seed Round",Venture Capital,"[australia, brazil, canada, china, denmark, es...","[accelerator/incubator, early stage vc, seed r...","[agriculture, business products and services (..."
...,...,...,...,...,...,...,...,...,...,...
155339,ECS Tuning,Imran Jooma,Manufacturer and distributor of automotive par...,,"Commercial Products, Transportation","Add-on, Buyout/LBO, Merger/Acquisition",0,[],"[add-on, buyout/lbo, merger/acquisition]","[commercial products, transportation]"
155340,ECSEL JU,0,,,"Semiconductors, Software",0,0,[],[0],"[semiconductors, software]"
155341,Ecster,0,Operator of payment solutions for both busines...,,0,Merger/Acquisition,0,[],[merger/acquisition],[0]
155342,ECU Health,Michael Waldrum,,,0,Merger/Acquisition,0,[],[merger/acquisition],[0]


Step 2: checking all other possible duplications (rows with exactly same row values --> found to be 590 number of rows). 

In [54]:
# If values in a column conatains unhashable data e.g. list / ndarray, formula will return error --> last three columns
# Thus, only use columns with hashable types --> subset of other columns
# This formula will return a series of boolean values indicating whether each row is a duplicate, True = Duplicate

check_duplicates = combined_df.duplicated(subset=['Investors', 'Primary Contact', 'Description', 'Geography', 'Preferred Industry', 'Preferred Investment Type', 'Primary Investor Type'])
print( check_duplicates )

print( check_duplicates.sum() )           # Count of duplicates --> 590

0         False
1         False
2         False
3         False
4         False
          ...  
155339    False
155340    False
155341    False
155342    False
155343    False
Length: 155344, dtype: bool
590


In [55]:
duplicates_index_number = check_duplicates[check_duplicates == True]
duplicates_index_number.index

Index([ 11372,  26404,  26415,  26517,  32353,  32422,  32967,  33122,  33561,
        34570,
       ...
       153768, 153769, 153774, 153861, 153899, 154109, 154162, 154866, 154867,
       154873],
      dtype='int64', length=590)

The variable "combined_df" here is updated with all duplicates removed.

In [56]:
combined_df.drop(index = duplicates_index_number.index, inplace=True)  # Drop duplicates
combined_df

,Investors,Primary Contact,Description,Geography,Preferred Industry,Preferred Investment Type,Primary Investor Type,geography_tags,preferred_investment_type_tags,preferred_industry_tags
0,Techstars,David Cohen,"Founded in 26, Techstars is an accelerator bas...","Africa, Americas, Asia, Canada, Middle East, O...","Beverages, Computer Hardware, Education and Tr...","Accelerator/Incubator, Early Stage VC, Later S...",Accelerator/Incubator,"[africa, americas, asia, canada, middle east, ...","[accelerator/incubator, early stage vc, later ...","[beverages, computer hardware, education and t..."
1,Y Combinator,David Lieb,"Founded in 25, Y Combinator is an accelerator ...","Africa, Americas, Asia, Europe, Oceania, Unite...","Biotechnology, Commercial Transportation, Comm...","Accelerator/Incubator, Early Stage VC, Later S...",Accelerator/Incubator,"[africa, americas, asia, europe, oceania, unit...","[accelerator/incubator, early stage vc, later ...","[biotechnology, commercial transportation, com..."
2,Plug and Play Tech Center,Marc Steiner,"Founded in 26, Plug and Play Tech Center is an...","Africa, Americas, Asia, Canada, Europe, Middle...","Aerospace and Defense, Animal Husbandry, Aquac...","Accelerator/Incubator, Early Stage VC, Later S...",Accelerator/Incubator,"[africa, americas, asia, canada, europe, middl...","[accelerator/incubator, early stage vc, later ...","[aerospace and defense, animal husbandry, aqua..."
3,Gaingels,Paul Grossinger,"Founded in 214, Gaingels is a venture capital ...","Africa, Americas, Asia, Canada, Europe, Middle...","Business Products and Services (B2B), Consumer...","Early Stage VC, Later Stage VC, PE Growth/Expa...",Venture Capital,"[africa, americas, asia, canada, europe, middl...","[early stage vc, later stage vc, pe growth/exp...","[business products and services (b2b), consume..."
4,Antler,Magnus Grimeland,"Founded in 217, Antler is a venture capital in...","Australia, Brazil, Canada, China, Denmark, Est...","Agriculture, Business Products and Services (B...","Accelerator/Incubator, Early Stage VC, Seed Round",Venture Capital,"[australia, brazil, canada, china, denmark, es...","[accelerator/incubator, early stage vc, seed r...","[agriculture, business products and services (..."
...,...,...,...,...,...,...,...,...,...,...
155339,ECS Tuning,Imran Jooma,Manufacturer and distributor of automotive par...,,"Commercial Products, Transportation","Add-on, Buyout/LBO, Merger/Acquisition",0,[],"[add-on, buyout/lbo, merger/acquisition]","[commercial products, transportation]"
155340,ECSEL JU,0,,,"Semiconductors, Software",0,0,[],[0],"[semiconductors, software]"
155341,Ecster,0,Operator of payment solutions for both busines...,,0,Merger/Acquisition,0,[],[merger/acquisition],[0]
155342,ECU Health,Michael Waldrum,,,0,Merger/Acquisition,0,[],[merger/acquisition],[0]


**2. Handle Missing Vlue**

In [57]:
# import os
# Define the path to your desktop
#desktop_path = os.path.expanduser("~/Desktop/")

# Define the file path for the CSV file on your desktop
#csv_file_path = os.path.join(desktop_path, 'output.csv')

# Export the DataFrame to a CSV file on your desktop
#combined_df.to_csv(csv_file_path, index=False)

In [58]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 154754 entries, 0 to 155343
Data columns (total 10 columns):
 #   Column                          Non-Null Count   Dtype 
---  ------                          --------------   ----- 
 0   Investors                       154754 non-null  object
 1   Primary Contact                 154753 non-null  object
 2   Description                     154754 non-null  object
 3   Geography                       154754 non-null  object
 4   Preferred Industry              154740 non-null  object
 5   Preferred Investment Type       154174 non-null  object
 6   Primary Investor Type           154091 non-null  object
 7   geography_tags                  154754 non-null  object
 8   preferred_investment_type_tags  154754 non-null  object
 9   preferred_industry_tags         154754 non-null  object
dtypes: object(10)
memory usage: 13.0+ MB


In [59]:
combined_df.drop(columns=['Primary Contact'], inplace=True, errors='ignore')

In [60]:
print(combined_df)

                        Investors  \
0                       Techstars   
1                    Y Combinator   
2       Plug and Play Tech Center   
3                        Gaingels   
4                          Antler   
...                           ...   
155339                 ECS Tuning   
155340                   ECSEL JU   
155341                     Ecster   
155342                 ECU Health   
155343              ECU Worldwide   

                                              Description  \
0       Founded in 26, Techstars is an accelerator bas...   
1       Founded in 25, Y Combinator is an accelerator ...   
2       Founded in 26, Plug and Play Tech Center is an...   
3       Founded in 214, Gaingels is a venture capital ...   
4       Founded in 217, Antler is a venture capital in...   
...                                                   ...   
155339  Manufacturer and distributor of automotive par...   
155340                                                      
155341

In [61]:
combined_df.isnull().sum()

Investors                           0
Description                         0
Geography                           0
Preferred Industry                 14
Preferred Investment Type         580
Primary Investor Type             663
geography_tags                      0
preferred_investment_type_tags      0
preferred_industry_tags             0
dtype: int64

In [62]:
drop_df = combined_df.dropna()

In [63]:
drop_df

,Investors,Description,Geography,Preferred Industry,Preferred Investment Type,Primary Investor Type,geography_tags,preferred_investment_type_tags,preferred_industry_tags
0,Techstars,"Founded in 26, Techstars is an accelerator bas...","Africa, Americas, Asia, Canada, Middle East, O...","Beverages, Computer Hardware, Education and Tr...","Accelerator/Incubator, Early Stage VC, Later S...",Accelerator/Incubator,"[africa, americas, asia, canada, middle east, ...","[accelerator/incubator, early stage vc, later ...","[beverages, computer hardware, education and t..."
1,Y Combinator,"Founded in 25, Y Combinator is an accelerator ...","Africa, Americas, Asia, Europe, Oceania, Unite...","Biotechnology, Commercial Transportation, Comm...","Accelerator/Incubator, Early Stage VC, Later S...",Accelerator/Incubator,"[africa, americas, asia, europe, oceania, unit...","[accelerator/incubator, early stage vc, later ...","[biotechnology, commercial transportation, com..."
2,Plug and Play Tech Center,"Founded in 26, Plug and Play Tech Center is an...","Africa, Americas, Asia, Canada, Europe, Middle...","Aerospace and Defense, Animal Husbandry, Aquac...","Accelerator/Incubator, Early Stage VC, Later S...",Accelerator/Incubator,"[africa, americas, asia, canada, europe, middl...","[accelerator/incubator, early stage vc, later ...","[aerospace and defense, animal husbandry, aqua..."
3,Gaingels,"Founded in 214, Gaingels is a venture capital ...","Africa, Americas, Asia, Canada, Europe, Middle...","Business Products and Services (B2B), Consumer...","Early Stage VC, Later Stage VC, PE Growth/Expa...",Venture Capital,"[africa, americas, asia, canada, europe, middl...","[early stage vc, later stage vc, pe growth/exp...","[business products and services (b2b), consume..."
4,Antler,"Founded in 217, Antler is a venture capital in...","Australia, Brazil, Canada, China, Denmark, Est...","Agriculture, Business Products and Services (B...","Accelerator/Incubator, Early Stage VC, Seed Round",Venture Capital,"[australia, brazil, canada, china, denmark, es...","[accelerator/incubator, early stage vc, seed r...","[agriculture, business products and services (..."
...,...,...,...,...,...,...,...,...,...
155339,ECS Tuning,Manufacturer and distributor of automotive par...,,"Commercial Products, Transportation","Add-on, Buyout/LBO, Merger/Acquisition",0,[],"[add-on, buyout/lbo, merger/acquisition]","[commercial products, transportation]"
155340,ECSEL JU,,,"Semiconductors, Software",0,0,[],[0],"[semiconductors, software]"
155341,Ecster,Operator of payment solutions for both busines...,,0,Merger/Acquisition,0,[],[merger/acquisition],[0]
155342,ECU Health,,,0,Merger/Acquisition,0,[],[merger/acquisition],[0]


In [64]:
drop_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 154087 entries, 0 to 155343
Data columns (total 9 columns):
 #   Column                          Non-Null Count   Dtype 
---  ------                          --------------   ----- 
 0   Investors                       154087 non-null  object
 1   Description                     154087 non-null  object
 2   Geography                       154087 non-null  object
 3   Preferred Industry              154087 non-null  object
 4   Preferred Investment Type       154087 non-null  object
 5   Primary Investor Type           154087 non-null  object
 6   geography_tags                  154087 non-null  object
 7   preferred_investment_type_tags  154087 non-null  object
 8   preferred_industry_tags         154087 non-null  object
dtypes: object(9)
memory usage: 11.8+ MB


In [65]:
drop_df.columns

Index(['Investors', 'Description', 'Geography', 'Preferred Industry',
       'Preferred Investment Type', 'Primary Investor Type', 'geography_tags',
       'preferred_investment_type_tags', 'preferred_industry_tags'],
      dtype='object')

In [66]:
drop_df.isin(['', '0', '[]', '[0]'])

,Investors,Description,Geography,Preferred Industry,Preferred Investment Type,Primary Investor Type,geography_tags,preferred_investment_type_tags,preferred_industry_tags
0,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
155339,False,False,True,False,False,True,True,False,False
155340,False,True,True,False,True,True,True,False,False
155341,False,False,True,True,False,True,True,False,False
155342,False,True,True,True,False,True,True,False,False


In [67]:
drop_df.isin(['', '0', '[]', '[0]']).any(axis=1)

0         False
1         False
2         False
3         False
4         False
          ...  
155339     True
155340     True
155341     True
155342     True
155343     True
Length: 154087, dtype: bool

In [68]:
~drop_df.isin(['', '0', '[]', '[0]']).any(axis=1)

0          True
1          True
2          True
3          True
4          True
          ...  
155339    False
155340    False
155341    False
155342    False
155343    False
Length: 154087, dtype: bool

In [69]:
drop_df = drop_df[~drop_df.isin(['', '0', '[]', '[0]']).any(axis=1)]


In [70]:
data1 = {
    'Category': ['A', 'B', 'A', 'C', 'B']
}

df3 = pd.DataFrame(data1)

# Convert the categorical column 'Category' into dummy variables
dummy_df = pd.get_dummies(df3['Category'], prefix='Category')

# Concatenate the dummy variables with the original DataFrame
df3 = pd.concat([df3, dummy_df], axis=1)

df3

,Category,Category_A,Category_B,Category_C
0,A,True,False,False
1,B,False,True,False
2,A,True,False,False
3,C,False,False,True
4,B,False,True,False


In [71]:
drop_df

,Investors,Description,Geography,Preferred Industry,Preferred Investment Type,Primary Investor Type,geography_tags,preferred_investment_type_tags,preferred_industry_tags
0,Techstars,"Founded in 26, Techstars is an accelerator bas...","Africa, Americas, Asia, Canada, Middle East, O...","Beverages, Computer Hardware, Education and Tr...","Accelerator/Incubator, Early Stage VC, Later S...",Accelerator/Incubator,"[africa, americas, asia, canada, middle east, ...","[accelerator/incubator, early stage vc, later ...","[beverages, computer hardware, education and t..."
1,Y Combinator,"Founded in 25, Y Combinator is an accelerator ...","Africa, Americas, Asia, Europe, Oceania, Unite...","Biotechnology, Commercial Transportation, Comm...","Accelerator/Incubator, Early Stage VC, Later S...",Accelerator/Incubator,"[africa, americas, asia, europe, oceania, unit...","[accelerator/incubator, early stage vc, later ...","[biotechnology, commercial transportation, com..."
2,Plug and Play Tech Center,"Founded in 26, Plug and Play Tech Center is an...","Africa, Americas, Asia, Canada, Europe, Middle...","Aerospace and Defense, Animal Husbandry, Aquac...","Accelerator/Incubator, Early Stage VC, Later S...",Accelerator/Incubator,"[africa, americas, asia, canada, europe, middl...","[accelerator/incubator, early stage vc, later ...","[aerospace and defense, animal husbandry, aqua..."
3,Gaingels,"Founded in 214, Gaingels is a venture capital ...","Africa, Americas, Asia, Canada, Europe, Middle...","Business Products and Services (B2B), Consumer...","Early Stage VC, Later Stage VC, PE Growth/Expa...",Venture Capital,"[africa, americas, asia, canada, europe, middl...","[early stage vc, later stage vc, pe growth/exp...","[business products and services (b2b), consume..."
4,Antler,"Founded in 217, Antler is a venture capital in...","Australia, Brazil, Canada, China, Denmark, Est...","Agriculture, Business Products and Services (B...","Accelerator/Incubator, Early Stage VC, Seed Round",Venture Capital,"[australia, brazil, canada, china, denmark, es...","[accelerator/incubator, early stage vc, seed r...","[agriculture, business products and services (..."
...,...,...,...,...,...,...,...,...,...
155260,Ecosphere Ventures,"Founded in 222, Ecosphere Ventures is a ventur...",United States,"Building Products, Commercial Transportation, ...","Early Stage VC, Seed Round",<span>0.00 - 1.00</span>,[united states],"[early stage vc, seed round]","[building products, commercial transportation,..."
155262,Ecostar (Accelerator),"Founded in 216, Ecostar is an accelerator inve...",Europe,"Agriculture, Consumer Products and Services (B...","Accelerator/Incubator, Debt Refinancing, Early...",<span>0.16</span>,[europe],"[accelerator/incubator, debt refinancing, earl...","[agriculture, consumer products and services (..."
155270,Ecosystem Ventures (India),"Founded in 218, Ecosystem Ventures is an angel...",Asia,"Business Products and Services (B2B), Energy, ...","Early Stage VC, Later Stage VC, Seed Round",<span>0.25 - 1.00</span>,[asia],"[early stage vc, later stage vc, seed round]","[business products and services (b2b), energy,..."
155314,Ecphora Capital,"Founded in 222, Ecphora Capital is a venture c...",Maryland,"Biotechnology, Healthcare Devices and Supplies","Early Stage VC, Seed Round",<span>0.50 - 4.00</span>,[maryland],"[early stage vc, seed round]","[biotechnology, healthcare devices and supplies]"


In [ ]:
drop_df.info()

# import os
# Define the path to your desktop
desktop_path = os.path.expanduser("~/Desktop/")

# Define the file path for the CSV file on your desktop
csv_file_path = os.path.join(desktop_path, 'output2.csv')

# Export the DataFrame to a CSV file on your desktop
drop_df.to_csv(csv_file_path, index=False)

**3. Remove shifted data.**

Check shift by checking "www." in the column "Description"

In [73]:
df_removed_shifted = drop_df[['Description']][drop_df['Description'].str.contains('www.')]
df_removed_shifted

,Description
118,www.hf.com
251,www.nhqv.com
338,www.tdpfund.com
551,www.andlinger.com
1672,www.jandjgroup.com
...,...
153206,www.dunross.com.cy
153629,www.dynamicpt.com
153758,www.ea-companies.com
154717,www.vertexventures.sg


In [74]:
index_list_removed_shifted = df_removed_shifted.index.tolist()
index_list_removed_shifted

[118,
 251,
 338,
 551,
 1672,
 1867,
 1873,
 1950,
 1959,
 1960,
 2437,
 5767,
 6040,
 6983,
 8058,
 8590,
 9463,
 9474,
 9475,
 9536,
 10128,
 11483,
 11612,
 11826,
 12002,
 12370,
 13490,
 14180,
 14196,
 14418,
 14590,
 14766,
 15135,
 15141,
 15438,
 15733,
 17349,
 17360,
 17959,
 18049,
 18062,
 18170,
 18191,
 18576,
 18787,
 19131,
 19226,
 19972,
 20308,
 21556,
 22484,
 23489,
 24690,
 25442,
 26351,
 27752,
 29005,
 30053,
 30201,
 30466,
 30870,
 32089,
 32555,
 33316,
 36283,
 38338,
 38471,
 38614,
 39302,
 39701,
 39974,
 40586,
 40597,
 40656,
 41211,
 41622,
 41973,
 42174,
 42324,
 42890,
 44752,
 45274,
 45593,
 46461,
 47375,
 47376,
 48286,
 49408,
 49612,
 50268,
 51085,
 52212,
 52578,
 53123,
 53147,
 53535,
 54936,
 55367,
 55590,
 55608,
 55955,
 56121,
 56216,
 56271,
 56625,
 57065,
 57096,
 57695,
 59376,
 59607,
 60028,
 60340,
 60958,
 61254,
 61414,
 62254,
 62735,
 63417,
 63458,
 64478,
 65321,
 65413,
 66289,
 67823,
 68437,
 70917,
 70923,
 73185,


Drop shifted rows and store in a new dataframe "drop_df_removed_shift"

In [75]:
drop_df_removed_shift = drop_df.drop(index= index_list_removed_shifted )
drop_df_removed_shift

,Investors,Description,Geography,Preferred Industry,Preferred Investment Type,Primary Investor Type,geography_tags,preferred_investment_type_tags,preferred_industry_tags
0,Techstars,"Founded in 26, Techstars is an accelerator bas...","Africa, Americas, Asia, Canada, Middle East, O...","Beverages, Computer Hardware, Education and Tr...","Accelerator/Incubator, Early Stage VC, Later S...",Accelerator/Incubator,"[africa, americas, asia, canada, middle east, ...","[accelerator/incubator, early stage vc, later ...","[beverages, computer hardware, education and t..."
1,Y Combinator,"Founded in 25, Y Combinator is an accelerator ...","Africa, Americas, Asia, Europe, Oceania, Unite...","Biotechnology, Commercial Transportation, Comm...","Accelerator/Incubator, Early Stage VC, Later S...",Accelerator/Incubator,"[africa, americas, asia, europe, oceania, unit...","[accelerator/incubator, early stage vc, later ...","[biotechnology, commercial transportation, com..."
2,Plug and Play Tech Center,"Founded in 26, Plug and Play Tech Center is an...","Africa, Americas, Asia, Canada, Europe, Middle...","Aerospace and Defense, Animal Husbandry, Aquac...","Accelerator/Incubator, Early Stage VC, Later S...",Accelerator/Incubator,"[africa, americas, asia, canada, europe, middl...","[accelerator/incubator, early stage vc, later ...","[aerospace and defense, animal husbandry, aqua..."
3,Gaingels,"Founded in 214, Gaingels is a venture capital ...","Africa, Americas, Asia, Canada, Europe, Middle...","Business Products and Services (B2B), Consumer...","Early Stage VC, Later Stage VC, PE Growth/Expa...",Venture Capital,"[africa, americas, asia, canada, europe, middl...","[early stage vc, later stage vc, pe growth/exp...","[business products and services (b2b), consume..."
4,Antler,"Founded in 217, Antler is a venture capital in...","Australia, Brazil, Canada, China, Denmark, Est...","Agriculture, Business Products and Services (B...","Accelerator/Incubator, Early Stage VC, Seed Round",Venture Capital,"[australia, brazil, canada, china, denmark, es...","[accelerator/incubator, early stage vc, seed r...","[agriculture, business products and services (..."
...,...,...,...,...,...,...,...,...,...
155260,Ecosphere Ventures,"Founded in 222, Ecosphere Ventures is a ventur...",United States,"Building Products, Commercial Transportation, ...","Early Stage VC, Seed Round",<span>0.00 - 1.00</span>,[united states],"[early stage vc, seed round]","[building products, commercial transportation,..."
155262,Ecostar (Accelerator),"Founded in 216, Ecostar is an accelerator inve...",Europe,"Agriculture, Consumer Products and Services (B...","Accelerator/Incubator, Debt Refinancing, Early...",<span>0.16</span>,[europe],"[accelerator/incubator, debt refinancing, earl...","[agriculture, consumer products and services (..."
155270,Ecosystem Ventures (India),"Founded in 218, Ecosystem Ventures is an angel...",Asia,"Business Products and Services (B2B), Energy, ...","Early Stage VC, Later Stage VC, Seed Round",<span>0.25 - 1.00</span>,[asia],"[early stage vc, later stage vc, seed round]","[business products and services (b2b), energy,..."
155314,Ecphora Capital,"Founded in 222, Ecphora Capital is a venture c...",Maryland,"Biotechnology, Healthcare Devices and Supplies","Early Stage VC, Seed Round",<span>0.50 - 4.00</span>,[maryland],"[early stage vc, seed round]","[biotechnology, healthcare devices and supplies]"


**4. Work on One Hot Encoding**

In [76]:
geography_tags_list = drop_df_removed_shift['geography_tags'].explode().dropna().unique().tolist()
print("Geography Tags:", geography_tags_list)

df_to_excel = pd.DataFrame(geography_tags_list)
df_to_excel

Geography Tags: ['africa', 'americas', 'asia', 'canada', 'middle east', 'oceania', 'united kingdom', 'united states', 'europe', 'australia', 'brazil', 'china', 'denmark', 'estonia', 'france', 'germany', 'india', 'indonesia', 'japan', 'kenya', 'malaysia', 'netherlands', 'norway', 'pakistan', 'philippines', 'portugal', 'singapore', 'south korea', 'spain', 'sweden', 'thailand', 'united arab emirates', 'vietnam', 'north america', 'central america', 'south america', 'israel', 'ireland', 'bay area', 'mexico', 'new york', 'new york metro', 'costa rica', 'cuba', 'dominica', 'el salvador', 'east asia', 'mid atlantic', 'northeast', 'south asia', 'luxembourg', 'hong kong', 'southeast asia', 'taiwan', 'andorra', 'austria', 'belgium', 'czech republic', 'hungary', 'liechtenstein', 'monaco', 'northern europe', 'poland', 'slovakia', 'slovenia', 'switzerland', 'southern europe', 'western europe', 'midwest', 'southeast', 'west coast', 'eastern europe', 'south', 'new zealand', 'indiana', 'california', 'f

,0
0,africa
1,americas
2,asia
3,canada
4,middle east
...,...
573,chad
574,serves as chairman at hivello. he is a co-foun...
575,serves as chairman at red swan ventures. he is...
576,glass bottles


In [77]:
df_to_excel.columns = ['Location']
df_to_excel

,Location
0,africa
1,americas
2,asia
3,canada
4,middle east
...,...
573,chad
574,serves as chairman at hivello. he is a co-foun...
575,serves as chairman at red swan ventures. he is...
576,glass bottles


In [78]:
%pip install pycountry

import pandas as pd
import zipfile
import io
import requests
import pycountry

   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   --- ------------------------------------ 0.5/6.3 MB 2.4 MB/s eta 0:00:03
   ------ --------------------------------- 1.0/6.3 MB 2.6 MB/s eta 0:00:02
   --------- ------------------------------ 1.6/6.3 MB 2.4 MB/s eta 0:00:02
   ------------- -------------------------- 2.1/6.3 MB 2.5 MB/s eta 0:00:02
   -------------- ------------------------- 2.4/6.3 MB 2.4 MB/s eta 0:00:02
   ------------------ --------------------- 2.9/6.3 MB 2.4 MB/s eta 0:00:02
   --------------------- ------------------ 3.4/6.3 MB 2.4 MB/s eta 0:00:02
   ------------------------ --------------- 3.9/6.3 MB 2.4 MB/s eta 0:00:01
   ----------------------------- ---------- 4.7/6.3 MB 2.5 MB/s eta 0:00:01
   ------------------------------- -------- 5.0/6.3 MB 2.5 MB/s eta 0:00:01
   ---------------------------------- ----- 5.5/6.3 MB 2.4 MB/s eta 0:00:01
   -------------------------------------- - 6.0/6.3 MB 2.4 MB/s eta 0:00:01
   ----------------

In [79]:
# Step 1: 下載並讀取 GeoNames 的城市資料
url = "http://download.geonames.org/export/dump/cities15000.zip"
response = requests.get(url)

if response.status_code == 200:
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        with z.open('cities15000.txt') as f:
            columns = [
                'geonameid', 'name', 'asciiname', 'alternatenames', 'latitude', 'longitude',
                'feature_class', 'feature_code', 'country_code', 'cc2', 'admin1_code',
                'admin2_code', 'admin3_code', 'admin4_code', 'population', 'elevation',
                'dem', 'timezone', 'modification_date'
            ]
            df_geo = pd.read_csv(f, sep='\t', header=None, names=columns)
else:
    raise Exception("GeoNames 資料下載失敗，請檢查網路。")

# Step 2: 建立 地名 ➜ 國家名稱 的對照字典
def get_country_name(code):
    try:
        return pycountry.countries.get(alpha_2=code).name
    except:
        return None

df_geo['country_name'] = df_geo['country_code'].apply(get_country_name)

# 使用 asciiname 建立城市/地區 對應國家名
location_to_country = dict(zip(df_geo['asciiname'].str.lower(), df_geo['country_name']))

# 加上所有國家名稱的直接對應（像 "Germany" ➜ "Germany"）
for country in pycountry.countries:
    location_to_country[country.name.lower()] = country.name

# Step 3: 建立轉換函數
def standardize_location(value):
    if pd.isna(value):
        return 'N/A'
    name = value.strip().lower()
    return location_to_country.get(name, 'N/A')

# Step 4: 應用到你的 df_to_excel
df_to_excel['Standardized_Country'] = df_to_excel['Location'].apply(standardize_location)

# Step 5: 預覽結果（可省略）
print(df_to_excel[['Location', 'Standardized_Country']])

# Step 6: 儲存結果到 Excel
df_to_excel.to_excel('standardized_locations.xlsx', index=False)

                                              Location Standardized_Country
0                                               africa                  N/A
1                                             americas                  N/A
2                                                 asia          Philippines
3                                               canada               Canada
4                                          middle east                  N/A
..                                                 ...                  ...
573                                               chad                 Chad
574  serves as chairman at hivello. he is a co-foun...                  N/A
575  serves as chairman at red swan ventures. he is...                  N/A
576                                      glass bottles                  N/A
577                              mining and industrial                  N/A

[578 rows x 2 columns]
